In [1]:
import numpy as np

In [2]:
USER_NUM=943
ITEM_NUM=1682
FACTOR=50

In [3]:
P=np.random.rand(USER_NUM,FACTOR)/(FACTOR**0.5)
Q=np.random.rand(ITEM_NUM,FACTOR)/(FACTOR**0.5)
print(P.shape)
print(Q.shape)

(943, 50)
(1682, 50)


In [4]:
def load_data(path, flag='train', sep='\t',size=0.8):
    with open('./{}'.format(path),'r') as file:
        for index,line in enumerate(file):
            if index ==0:
                continue
            rand_num = np.random.rand()
            if flag == 'train' :
                if rand_num < size :
                    u,i,r,t=line.strip('\r\n').split(sep)
                    yield (int(u)-1,int(i)-1,float(r))
            else:
                if rand_num >= size:
                    u,i,r,t=line.strip('\r\n').split(sep)
                    yield (int(u)-1,int(i)-1,float(r))

In [5]:
def train(train_data, random_state=0,epochs=5,theta=1e-4,alpha=0.02,beta=0.02):
    current_e = 0.0
    old_e = 0.0
    cost_of_epoch = []
    np.random.seed(random_state)
    for epoch in range(epochs):
        for index,data in enumerate(train_data):
            u,i,rating = data
            predict_rating = np.dot(P[u],Q[i].T)
            err = rating - predict_rating
            current_e += pow(err,2)
            P[u] += alpha * (err * Q[i] - beta*P[u])
            Q[i] += alpha * (err * P[u] - beta*Q[i])
            current_e+=(beta/2)*(sum(pow(P[u],2))+sum(pow(Q[i],2))) #正则项
        cost_of_epoch.append(current_e)
        print('cost is {}'.format(current_e))
        if abs(current_e - old_e) < theta:
            break
        old_e=current_e
        alpha*=0.9
train_data = load_data('./u.data')
train(train_data)

cost is 196700.96286041357
cost is 196700.96286041357


In [6]:
def test_rmse(test_data):
    rmse = 0.0
    num = 0
    for index,data in enumerate(test_data):
        u,i,rating = data
        num = index+1
        predict_rating = np.dot(P[u], Q[i])
        rmse += np.power(rating-predict_rating,2)
    rmse=(rmse/num)**0.5
    return rmse
train_data = load_data('./u.data',flag='test')
print(test_rmse(train_data))

1.0050975636907462
